In [1]:
import os
import torch
import librosa
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchaudio import load
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, roc_curve
import torch.optim as optim

In [2]:
class AudioDataset(Dataset):
    def __init__(self, root):
        self.root = root
        self.classes, self.class_to_idx = self._find_classes()
        self.samples = self._make_dataset()

    def _find_classes(self):
        classes = [d.name for d in os.scandir(self.root) if d.is_dir()]
        classes.sort()
        class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
        return classes, class_to_idx

    def _make_dataset(self):
        samples = []
        for target_class in self.classes:
            class_index = self.class_to_idx[target_class]
            target_dir = os.path.join(self.root, target_class)
            for root_dir, _, file_names in os.walk(target_dir):
                for file_name in file_names:
                    if file_name.endswith('.wav') or file_name.endswith('.mp3') or file_name.endswith('.ogg'):
                        file_path = os.path.join(root_dir, file_name)
                        samples.append((file_path, class_index))
        return samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        audio_path, class_index = self.samples[idx]
        # Load the audio file and preprocess
        waveform, _ = load(audio_path)
        waveform = self._preprocess_audio(waveform)
        return waveform, class_index

    def _preprocess_audio(self, waveform):
        waveform = waveform.numpy()[0]  # Convert tensor to numpy array
        max_len = 64600
        if waveform.shape[0] >= max_len:
            return waveform[:max_len]
        else:
            num_repeats = int(max_len / waveform.shape[0]) + 1
            padded_waveform = np.tile(waveform, (1, num_repeats))[:, :max_len][0]
            return padded_waveform


In [3]:
# Define the root directory where your data is stored
root = "/teamspace/studios/this_studio/SpeechAssign3/for-2seconds"  # Replace this with the path to your data folder

# Create datasets
train_dataset = AudioDataset(root=os.path.join(root, "training"))
test_dataset = AudioDataset(root=os.path.join(root, "testing"))
validation_dataset = AudioDataset(root=os.path.join(root, "validation"))

In [4]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=6)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)
validation_loader = DataLoader(validation_dataset, batch_size=16, shuffle=False, num_workers=6)

In [5]:
#GPU device
device = 'cuda' if torch.cuda.is_available() else 'cpu'                  
print('Device: {}'.format(device))

# Load the SSL W2V model trained for LA and DF tracks
from model import Model

model = Model(None, device=device)
model = nn.DataParallel(model).to(device)
model.load_state_dict(torch.load('/teamspace/studios/this_studio/final_model.pth'))

model.eval()

Device: cuda


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


DataParallel(
  (module): Model(
    (ssl_model): SSLModel(
      (model): Wav2Vec2Model(
        (feature_extractor): ConvFeatureExtractionModel(
          (conv_layers): ModuleList(
            (0): Sequential(
              (0): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
              (1): Dropout(p=0.0, inplace=False)
              (2): Sequential(
                (0): TransposeLast()
                (1): Fp32LayerNorm((512,), eps=1e-05, elementwise_affine=True)
                (2): TransposeLast()
              )
              (3): GELU(approximate='none')
            )
            (1-4): 4 x Sequential(
              (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
              (1): Dropout(p=0.0, inplace=False)
              (2): Sequential(
                (0): TransposeLast()
                (1): Fp32LayerNorm((512,), eps=1e-05, elementwise_affine=True)
                (2): TransposeLast()
              )
              (3): GELU(approximate='none')
            )
     

In [7]:
from sklearn.metrics import roc_auc_score, roc_curve
from scipy.optimize import brentq
from scipy.interpolate import interp1d
import numpy as np
import torch
from tqdm import tqdm

# Lists to store true labels and predicted scores
true_labels = []
predicted_scores = []

# Set the model to evaluation mode
model.eval()

# Iterate through the test_loader to get true labels and predicted scores
with torch.no_grad():
    for data, target in tqdm(test_loader, desc="Testing"):
        # Assuming your model outputs probabilities or scores
        output = model(data)
        predicted_scores.extend(output[:, 1].cpu().numpy())  # Probability of positive class
        true_labels.extend(target.cpu().numpy())

# Convert lists to numpy arrays
true_labels = np.array(true_labels)
predicted_scores = np.array(predicted_scores)

# Calculate AUC
auc_score = roc_auc_score(true_labels, predicted_scores)

# Calculate EER
fpr, tpr, thresholds = roc_curve(true_labels, predicted_scores, pos_label=1)
eer = brentq(lambda x: 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
threshold = interp1d(fpr, thresholds)(eer)

print("AUC:", auc_score)
print("EER:", eer)
print("Threshold at EER:", threshold)


Testing: 100%|██████████| 68/68 [00:23<00:00,  2.87it/s]

AUC: 0.3740133001730104
EER: 0.5772058823525775
Threshold at EER: 2.7146806716927294
